In [32]:
import pandas as pd
from os import path
import glob
from prophet import Prophet

In [33]:
folders=glob.glob('data\*')

In [34]:
btc_files=glob.glob(folders[0]+'\*')
eth_files=glob.glob(folders[1]+'\*')
ltc_files=glob.glob(folders[2]+'\*')

## Training data

1. BTC

In [35]:
btc=pd.read_csv(btc_files[0],header=1,index_col=1,parse_dates=['Date'])
btc=btc.resample('60T').mean()
btc.drop(['Unix Timestamp'],axis=1,inplace=True)
btc.index=btc.index.rename('datetime')

btc_series=btc[['Close']].copy()
btc_series.rename(columns={'Close':'btc'},inplace=True)
btc_series['ds']=btc_series.index.values

In [36]:
btc_series.to_csv('btc_hourly_close.csv')

2. Eth

In [37]:
eth=pd.read_csv(eth_files[0],header=1,index_col=1,parse_dates=['Date'])
eth=eth.resample('60T').mean()
eth.drop(['Unix Timestamp'],axis=1,inplace=True)
eth.index=eth.index.rename('datetime')

eth_series=eth[['Close']].copy()
eth_series.rename(columns={'Close':'eth'},inplace=True)
eth_series['ds']=eth_series.index.values

In [38]:
eth_series.to_csv('eth_hourly_close.csv')

3. LTC

In [39]:
ltc=pd.read_csv(ltc_files[0],header=1,index_col=1,parse_dates=['Date'])
ltc=ltc.resample('60T').mean()
ltc.drop(['Unix Timestamp'],axis=1,inplace=True)
ltc.index=ltc.index.rename('datetime')

ltc_series=ltc[['Close']].copy()
ltc_series.rename(columns={'Close':'ltc'},inplace=True)
ltc_series['ds']=ltc_series.index.values

In [40]:
ltc_series.to_csv('ltc_hourly_close.csv')

## Visulization of data

In [41]:
import plotly.express as px 

T=ltc_series['2020-02-01':'2020-04-01'] 

fig=px.line(x=T.index, y=T["ltc"],
 title='LiteCoin Closing Price',
 template='plotly_dark',
 labels={'y':'LiteCoin price in USD'},
 color_discrete_sequence=['orangered'])

fig.show()

In [42]:
df=pd.concat([ltc_series[['ltc']],btc_series[['btc']],eth_series[['eth']]],axis=1,join='inner')

In [43]:
df

,ltc,btc,eth
datetime,,,
2018-10-16 13:00:00,54.60,6443.96,204.55
2018-10-16 14:00:00,53.84,6435.04,204.51
2018-10-16 15:00:00,53.67,6434.98,204.35
2018-10-16 16:00:00,52.88,6409.53,203.90
2018-10-16 17:00:00,52.95,6422.10,203.55
...,...,...,...
2021-10-12 20:00:00,172.28,55376.80,3478.26
2021-10-12 21:00:00,171.99,56067.93,3484.75
2021-10-12 22:00:00,172.62,56409.77,3499.19


In [44]:
df.tail()

,ltc,btc,eth
datetime,,,
2021-10-12 20:00:00,172.28,55376.80,3478.26
2021-10-12 21:00:00,171.99,56067.93,3484.75
2021-10-12 22:00:00,172.62,56409.77,3499.19
2021-10-12 23:00:00,172.29,56019.08,3490.56
2021-10-13 00:00:00,172.94,56044.05,3500.17


In [45]:
start='2019-11-01'
end='2020-08-01'
T=df[start:end]
fig=px.line(T, title='Closing Price in USD',template='plotly_dark',labels={'value':'USD','variable':'Crypto assets','datetime':'Date'})
fig.data[0].name='LTC'
fig.data[1].name='BTC'
fig.data[2].name='ETH'

fig.show()

## Grab data from nomics API

In [108]:
import datetime
n = datetime.datetime.now(datetime.timezone.utc) - datetime.timedelta(4,5,6)
start=n.isoformat()

In [125]:
import urllib.parse


key='6a3b16cf358e1b21ed1ce04e12c415f7e1184dca'



getVars = {'key': key,'ids':'ETH,BTC','start': start}

url = 'https://api.nomics.com/v1/currencies/sparkline?'
request_url=url + urllib.parse.urlencode(getVars)
print(request_url)

https://api.nomics.com/v1/currencies/sparkline?key=6a3b16cf358e1b21ed1ce04e12c415f7e1184dca&ids=ETH%2CBTC&start=2021-10-09T22%3A04%3A23.223995%2B00%3A00


In [126]:
import requests
response = requests.get(request_url).json()

In [127]:
response

[{'currency': 'BTC',
  'timestamps': ['2021-10-10T00:00:00Z',
   '2021-10-11T00:00:00Z',
   '2021-10-12T00:00:00Z',
   '2021-10-13T00:00:00Z'],
  'prices': ['54887.89542969',
   '57683.84358316',
   '56195.96212781',
   '57001.82663275']},
 {'currency': 'ETH',
  'timestamps': ['2021-10-10T00:00:00Z',
   '2021-10-11T00:00:00Z',
   '2021-10-12T00:00:00Z',
   '2021-10-13T00:00:00Z'],
  'prices': ['3419.46595558',
   '3548.50819823',
   '3496.23370927',
   '3557.08865267']}]

In [124]:
response_df=pd.DataFrame.from_dict(response[1])
response_df

,currency,timestamps,prices
0,ETH,2021-10-10T00:00:00Z,3419.46595558
1,ETH,2021-10-11T00:00:00Z,3548.50819823
2,ETH,2021-10-12T00:00:00Z,3496.23370927
3,ETH,2021-10-13T00:00:00Z,3557.08865267


In [100]:
response

[{'id': 'BTC',
  'currency': 'BTC',
  'symbol': 'BTC',
  'name': 'Bitcoin',
  'logo_url': 'https://s3.us-east-2.amazonaws.com/nomics-api/static/images/currencies/btc.svg',
  'status': 'active',
  'price': '57623.71156218',
  'price_date': '2021-10-13T00:00:00Z',
  'price_timestamp': '2021-10-13T21:50:00Z',
  'circulating_supply': '18842993',
  'max_supply': '21000000',
  'market_cap': '1085803193600',
  'market_cap_dominance': '0.4118',
  'num_exchanges': '392',
  'num_pairs': '66146',
  'num_pairs_unmapped': '5169',
  'first_candle': '2011-08-18T00:00:00Z',
  'first_trade': '2011-08-18T00:00:00Z',
  'first_order_book': '2017-01-06T00:00:00Z',
  'rank': '1',
  'rank_delta': '0',
  'high': '63661.47450422',
  'high_timestamp': '2021-04-13T00:00:00Z',
  '1d': {'volume': '52319093688.00',
   'price_change': '1348.83960477',
   'price_change_pct': '0.0240',
   'volume_change': '-3122915556.09',
   'volume_change_pct': '-0.0563',
   'market_cap_change': '25472450102.74',
   'market_cap_chan

In [89]:
response_df.index

Index(['market_cap_change', 'market_cap_change_pct', 'price_change',
       'price_change_pct', 'volume', 'volume_change', 'volume_change_pct'],
      dtype='object')

In [80]:
response_df.circulating_supply

market_cap_change        18842618
market_cap_change_pct    18842618
price_change             18842618
price_change_pct         18842618
volume                   18842618
volume_change            18842618
volume_change_pct        18842618
Name: circulating_supply, dtype: object

In [93]:
response_df.T

,market_cap_change,market_cap_change_pct,price_change,price_change_pct,volume,volume_change,volume_change_pct
id,BTC,BTC,BTC,BTC,BTC,BTC,BTC
currency,BTC,BTC,BTC,BTC,BTC,BTC,BTC
symbol,BTC,BTC,BTC,BTC,BTC,BTC,BTC
name,Bitcoin,Bitcoin,Bitcoin,Bitcoin,Bitcoin,Bitcoin,Bitcoin
logo_url,https://s3.us-east-2.amazonaws.com/nomics-api/...,https://s3.us-east-2.amazonaws.com/nomics-api/...,https://s3.us-east-2.amazonaws.com/nomics-api/...,https://s3.us-east-2.amazonaws.com/nomics-api/...,https://s3.us-east-2.amazonaws.com/nomics-api/...,https://s3.us-east-2.amazonaws.com/nomics-api/...,https://s3.us-east-2.amazonaws.com/nomics-api/...
status,active,active,active,active,active,active,active
price,56355.78740059,56355.78740059,56355.78740059,56355.78740059,56355.78740059,56355.78740059,56355.78740059
price_date,2021-10-13T00:00:00Z,2021-10-13T00:00:00Z,2021-10-13T00:00:00Z,2021-10-13T00:00:00Z,2021-10-13T00:00:00Z,2021-10-13T00:00:00Z,2021-10-13T00:00:00Z
price_timestamp,2021-10-13T15:20:00Z,2021-10-13T15:20:00Z,2021-10-13T15:20:00Z,2021-10-13T15:20:00Z,2021-10-13T15:20:00Z,2021-10-13T15:20:00Z,2021-10-13T15:20:00Z
circulating_supply,18842618,18842618,18842618,18842618,18842618,18842618,18842618


In [4]:
import numpy as np
array1=np.array(list(range(8)))

In [5]:
array1

array([0, 1, 2, 3, 4, 5, 6, 7])

In [7]:
array1.reshape((-1,3))

ValueError: cannot reshape array of size 8 into shape (3)